In [ ]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as sm

# 1. Read microdata

In [ ]:
microdata = pd.read_excel("assets/excel/esa_microdata.xlsx", index_col=[0])

In [ ]:
display(microdata)

# 2. Divide into households groups by household income

In [ ]:
# The total number of households is equal to the number of rows in the microdata set
number_of_households = len(microdata.index)

In [ ]:
# Calculate the median income for the calculation of poverty indicators
median_income = microdata["einkommen_gesamt_jahr_eur"].median()

### Low income households

In [ ]:
low_income_threshold = 1/3  # Households in the bottom third of the income distribution"

# Sort the values by income and select the household income that is located at the threshold
low_income_threshold_income_eur = microdata.sort_values(by="einkommen_gesamt_jahr_eur").iloc[int(low_income_threshold*number_of_households)]["einkommen_gesamt_jahr_eur"]

# 3. Calculate price and income elasticities of demand

#### Define regression equation

In [ ]:
regression_formula = "np.log(stromverbrauch_jahr_kwh) ~ np.log(strompreis_cent_kwh) + np.log(einkommen_gesamt_jahr_eur) + anz_pers"

#### Create regression model

In [ ]:
ols_model = sm.ols(formula=regression_formula, data=microdata)

#### Fit model to data

In [ ]:
ols_model_results = ols_model.fit()

#### Show regression statistics

In [ ]:
display(ols_model_results.summary())

# 4. Calculate impact of relief measures

In [ ]:
price_discount_cent_kwh = -5 # €-ct/kWh

In [ ]:
electricity_compensation_eur = 350  # €/a

### For the case of a general price discount

In [ ]:
def calc_consumption_change_price_discount(old_consumption, price_elasticity, original_price, price_discount_cent_kwh):
    new_price = original_price + price_discount_cent_kwh
    return np.exp(price_elasticity*(np.log(new_price)-np.log(original_price))+np.log(old_consumption))

In [ ]:
# HIER MUSS DIE FUNCTION calc_consumption_change_price_discount AUFGERUFEN WERDEN
# DEM MICRODATENSATZ KÖNNEN ÜBER FOLGENDEN BEFEHL NEUE SPALTEN HINZUGEFÜGT WERDEN:
# microdata["neue_spalte"] = "xyz"

### For the case of an electricity cost compensation for low-income households

In [ ]:
def calc_consumption_change_compensation_payment(old_consumption, income_elasticity, old_income, new_income):
    return np.exp(income_elasticity*(np.log(new_income.astype(float))-np.log(old_income.astype(float)))+np.log(old_consumption.astype(float)))

In [ ]:
microdata["einkommen_gesamt_jahr_mit_zuschuss_eur"] = microdata["einkommen_gesamt_jahr_eur"]
microdata.loc[microdata["einkommen_gesamt_jahr_eur"] <= low_income_threshold_income_eur, "einkommen_gesamt_jahr_mit_zuschuss_eur"] += electricity_compensation_eur

In [ ]:
# HIER MUSS DIE FUNCTION calc_consumption_change_compensation_payment AUFGERUFEN WERDEN
# DEM MICRODATENSATZ KÖNNEN ÜBER FOLGENDEN BEFEHL NEUE SPALTEN HINZUGEFÜGT WERDEN:
# microdata["neue_spalte"] = "xyz"

# 5. Analyze impact of relief measures

### Without relief measures

In [ ]:
microdata["anteil_stromkosten"] = microdata["stromverbrauch_jahr_kwh"] * (microdata["strompreis_cent_kwh"]/100) / microdata["einkommen_gesamt_jahr_eur"]

In [ ]:
print("Durchschnittl. Anteil der Stromausgaben am Einkommen:",round(microdata["anteil_stromkosten"].mean()*100,2), "%")

In [ ]:
print("Gesamte Stromnachfrage:",round((microdata["stromverbrauch_jahr_kwh"]/1e6).sum(),2), "GWh")

In [ ]:
print("Armutsgefährdete Haushalte:",round(len(microdata.loc[microdata["einkommen_gesamt_jahr_eur"] <= 0.6*median_income].index)/number_of_households*100,2), "%")

### With price discount

In [ ]:
# ZUNÄCHST MUSS HIER EIN FIKTIVES NEUES EINKOMMEN BERECHNET WERDEN, 
# DASS KOSTENEINSPARUNGEN WIE EINE EINKOMMENSERHÖHUNG EINBERECHNET

### With compensation payment for low-income households